In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout, pout, pout],[pout, pin, pout, pout],[pout, pout, pin, pout],[pout, pout, pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs, seed=0)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10, 10, 10]
features_sizes = [8, 7, 6, 5]
pin = 0.5
pout = 0.1
seed = 0
alpha = 1e-2
lamda = 0#1e-3#1e-3
eta = 1e-2
d0 = min(features_sizes)
no_users = sum(cluster_sizes)
batch_size = 4
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

# Set a random seed for reproducibility
seed = 17
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         ... 0.09090909 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.1       ]
 ...
 [0.09090909 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.1        ... 0.         0.         0.        ]]
[[0.40353535 0.         0.         ... 0.09090909 0.         0.        ]
 [0.         0.53216783 0.         ... 0.         0.         0.        ]
 [0.         0.         0.13216783 ... 0.         0.         0.1       ]
 ...
 [0.09090909 0.         0.         ... 0.11796537 0.         0.        ]
 [0.         0.         0.         ... 0.         0.18100233 0.        ]
 [0.         0.         0.1        ... 0.         0.         0.1488345 ]]


In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 0.0, 5.0, -2.0, -3.0, 5.0, 2.0])
W2 = np.array([2.0, 2.0, 0.0, 5.0, 2.0, -3.0, -5.0])
W3 = np.array([2.0, 2.0, 0.0, 5.0, -2.0, -3.0])
W4 = np.array([2.0, 2.0, 0.0, -5.0, -2.0])
W = [W1, W2, W3, W4]
m = 8
n = [8, 7, 6, 5]
noise_sd = [0.001,0.01,0.1,1.0]

for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n[i]))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd[2])
        test_features = np.random.normal(loc=0.0, scale=1.0, size=(100, n[i]))
        test_label = np.dot(test_features, W[i]) + np.random.normal(0,0.05)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i]),
                'input_size': n[i],
                'test_features':test_features,
                'test_labels': test_label
            }
        count += 1

In [6]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [7]:
class MLP_Net(nn.Module):
    def __init__(self, input_size, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [8]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [9]:
model = MLP_Net(datapoints[19]["input_size"], user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=20, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for j, (x, y) in zip(range(1), dataloader):
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)

        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss.detach(), parameters_to_vector(model.parameters()).detach())
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

0 tensor(78.4140) tensor([-0.0497,  0.0265,  0.2496, -0.2843, -0.3558,  0.0373, -0.0888])
1 tensor(71.6787) tensor([-0.0047,  0.0180,  0.1399, -0.1483, -0.3487, -0.1515, -0.1227])
2 tensor(65.7366) tensor([ 0.0378,  0.0119,  0.0392, -0.0177, -0.3404, -0.3276, -0.1559])
3 tensor(60.4856) tensor([ 0.0780,  0.0081, -0.0532,  0.1079, -0.3309, -0.4921, -0.1883])
4 tensor(55.8371) tensor([ 0.1160,  0.0064, -0.1378,  0.2288, -0.3204, -0.6455, -0.2200])
5 tensor(51.7142) tensor([ 0.1521,  0.0067, -0.2153,  0.3452, -0.3089, -0.7888, -0.2510])
6 tensor(48.0500) tensor([ 0.1863,  0.0088, -0.2861,  0.4573, -0.2965, -0.9225, -0.2815])
7 tensor(44.7863) tensor([ 0.2187,  0.0125, -0.3507,  0.5654, -0.2833, -1.0473, -0.3113])
8 tensor(41.8727) tensor([ 0.2495,  0.0178, -0.4096,  0.6696, -0.2694, -1.1639, -0.3406])
9 tensor(39.2653) tensor([ 0.2787,  0.0246, -0.4633,  0.7703, -0.2548, -1.2727, -0.3693])
10 tensor(36.9259) tensor([ 0.3066,  0.0326, -0.5120,  0.8675, -0.2397, -1.3743, -0.3975])
11 tensor

In [10]:
parameters_to_vector(model.parameters())

tensor([ 1.1090,  1.5059, -0.5923,  4.3117,  1.0978, -2.8357, -2.0528],
       grad_fn=<CatBackward0>)

In [11]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [12]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [13]:
# Preparing projection matrices
models = [MLP_Net(input_size=datapoints[i]['input_size'], user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, models, first_run=True):
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        # Specify the dimensions of the rectangular matrix
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]

                        # Generate random values for the diagonal from a normal distribution
                        diag_values = 1.0 + 1.0 * torch.randn(min(row, column))

                        # Create a rectangular matrix with diagonal elements
                        mat = torch.diag(diag_values)

                        # If the matrix is larger than the diagonal vector, fill the remaining elements with zeros
                       
                        mat = torch.cat((mat, torch.zeros(row, column - row)), dim=1)

                        

                        # Append the matrix to the list
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights, models)



In [14]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["test_features"], dataset["test_labels"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [15]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [16]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(8.6603, dtype=torch.float64)

In [17]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(input_size=datapoints[i]['input_size'], user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(input_size=datapoints[i]['input_size'], user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

in_cluster_proj_norm = []
out_cluster_proj_norm = []
in_cluster_proj_diff_norm = []
out_cluster_proj_diff_norm = []
task_loss = {'0':[],
                '1':[],
                '2':[],
                '3':[]}
task_rel_error = {'0':[],
            '1':[],
            '2':[],
            '3':[]}

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    projected_weights = []
    update_ProjWeight(projection_list, projected_weights, models, first_run=False)
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            temp_mat = torch.outer(projected_weights[i][j] - projected_weights[j][i], weights).clone()
            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
    
    in_cluster_proj_norm_round = 0
    out_cluster_proj_norm_round = 0
    in_cluster_proj_diff_round = 0
    out_cluster_proj_diff_round = 0
    in_edges = 0
    out_edges = 0
    
    for i in range(no_users//4):
        for j in G.neighbors(i):
            if j < no_users//4:
                in_edges += 1
                in_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                in_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
            else:
                out_edges += 1
                out_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                out_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
    in_cluster_proj_norm.append(in_cluster_proj_norm_round / in_edges)
    out_cluster_proj_norm.append(out_cluster_proj_norm_round / out_edges)
    in_cluster_proj_diff_norm.append(in_cluster_proj_diff_round / in_edges)
    out_cluster_proj_diff_norm.append(out_cluster_proj_diff_round / out_edges)
            
                                         
                                              
    
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    per_task_loss = []
    per_task_rel_error = []
    
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
        if (k + 1) % 10 == 0:
            task_loss[str(k // 10)].append(sum(per_task_loss) / 10)
            task_rel_error[str(k // 10)].append(sum(per_task_rel_error) / 10)
            per_task_loss = []
            per_task_rel_error = []
        per_task_loss.append(g_loss)
        per_task_rel_error.append(rel_error(models[i]))
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 1/2000 [00:00<13:12,  2.52it/s]

Training_loss 67.75662,   Relative Error 0.91985


  0%|          | 2/2000 [00:00<13:27,  2.47it/s]

Training_loss 67.36572,   Relative Error 0.91744


  0%|          | 3/2000 [00:01<13:02,  2.55it/s]

...

Training_loss 10.89518,   Relative Error 0.39400
Training_loss 10.89061,   Relative Error 0.39397


100%|█████████▉| 1999/2000 [05:53<00:00,  6.82it/s]

Training_loss 10.88584,   Relative Error 0.39393
Training_loss 10.88805,   Relative Error 0.39388


100%|██████████| 2000/2000 [05:53<00:00,  5.66it/s]

Training_loss 10.88739,   Relative Error 0.39383


In [18]:
#plot.plot(test_loss)
print(parameters_to_vector(models[19].parameters()), W[1])

tensor([ 1.0331,  2.9987, -0.6963,  5.1755,  1.7718, -2.8098, -4.4091],
       grad_fn=<CatBackward0>) [ 2.  2.  0.  5.  2. -3. -5.]


In [19]:
for i in G.neighbors(3):
    print(i)

0
4
5
6
15
16
19
25
31


In [20]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

in_cluster_proj_norm = np.array(in_cluster_proj_norm)
out_cluster_proj_norm = np.array(out_cluster_proj_norm)
in_cluster_proj_diff_norm = np.array(in_cluster_proj_diff_norm)
out_cluster_proj_diff_norm = np.array(out_cluster_proj_diff_norm)


In [21]:
np.save( 'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)
np.save( 'in_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_norm)
np.save('out_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_norm)
np.save( 'in_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_diff_norm)
np.save('out_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_diff_norm)

In [22]:
'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'training_loss_sheave_fml0_pout0_1'

In [23]:
'relative_error_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'relative_error_sheave_fml0_pout0_1'

In [24]:
'in_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'in_cluster_proj_norm_sheave_fml0_pout0_1'

In [25]:
'out_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'out_cluster_proj_norm_sheave_fml0_pout0_1'

In [26]:
'in_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'in_cluster_proj_diff_norm_sheave_fml0_pout0_1'

In [27]:
'out_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'out_cluster_proj_diff_norm_sheave_fml0_pout0_1'

In [28]:
# task_loss dictionary
for key, value in task_loss.items():
    # Convert the list to a NumPy array
    array_loss = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('training_loss_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_loss)

# task_rel_error dictionary
for key, value in task_rel_error.items():
    # Convert the list to a NumPy array
    array_rel_error = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('relative_error_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_rel_error)

In [29]:
'relative_error_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_')

'relative_error_sheave_fml_task3_0_pout0_1'

In [30]:
task_loss['0']

[60.980960083007815,
 60.62915210723877,
 58.087818717956544,
 57.97616329193115,
 55.941121101379395,
 54.21936092376709,
 52.602606010437015,
 51.31578254699707,
 50.63236770629883,
 49.3741979598999,
 48.397734832763675,
 47.18665523529053,
 46.14659671783447,
 45.32335510253906,
 45.007185745239255,
 44.299446487426756,
 43.82498989105225,
 43.520512390136716,
 43.37413845062256,
 43.134957504272464,
 42.46207752227783,
 42.19277687072754,
 41.734449577331546,
 41.34343585968018,
 40.96257762908935,
 40.715349197387695,
 40.43331470489502,
 39.84458827972412,
 39.34904823303223,
 38.928687286376956,
 38.59447975158692,
 38.31453437805176,
 37.924856185913086,
 37.64488334655762,
 37.369524765014646,
 37.17091255187988,
 36.99143886566162,
 36.80692691802979,
 36.516333389282224,
 36.29406967163086,
 36.08881587982178,
 35.783667182922365,
 35.63506679534912,
 35.33243808746338,
 35.21474962234497,
 34.996220397949216,
 34.91371650695801,
 34.66103096008301,
 34.43311614990235,
 34.

In [31]:
task_loss['1']

[67.75662231445312,
 67.36572456359863,
 64.54202079772949,
 64.41795921325684,
 62.15680122375488,
 60.24373435974121,
 58.44734001159668,
 57.01753616333008,
 56.25818634033203,
 54.860219955444336,
 53.77526092529297,
 52.429616928100586,
 51.273996353149414,
 50.359283447265625,
 50.00798416137695,
 49.22160720825195,
 48.69443321228027,
 48.35612487792969,
 48.1934871673584,
 47.927730560302734,
 47.18008613586426,
 46.880863189697266,
 46.37161064147949,
 45.937150955200195,
 45.51397514343262,
 45.23927688598633,
 44.92590522766113,
 44.27176475524902,
 43.72116470336914,
 43.25409698486328,
 42.882755279541016,
 42.57170486450195,
 42.138729095458984,
 41.8276481628418,
 41.52169418334961,
 41.3010139465332,
 41.10159873962402,
 40.89658546447754,
 40.57370376586914,
 40.326744079589844,
 40.098684310913086,
 39.75963020324707,
 39.59451866149902,
 39.25826454162598,
 39.1274995803833,
 38.88468933105469,
 38.79301834106445,
 38.51225662231445,
 38.25901794433594,
 38.062924385

In [33]:
projection_list[0]

[0,
 0,
 0,
 tensor([[1.9701, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.5921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0033, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.1557, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 2.1053, 0.0000, 0.0000, 0.0000]],
        grad_fn=<AddBackward0>),
 tensor([[2.0324, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.9731, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.0599, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 1.7370, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 1.6656, 0.0000, 0.0000, 0.0000]],
        grad_fn=<AddBackward0>),
 0,
 tensor([[ 2.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.5508,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],


In [34]:
parameters_to_vector(models[0])

TypeError: 'MLP_Net' object is not iterable